In [1]:
#Import the required packages
#Import package pandas for data analysis
import pandas as pd

# Import package numpy for numeric computing
import numpy as np

# Import package matplotlib for visualisation/plotting
import matplotlib.pyplot as plt

#For showing plots directly in the notebook run the command below
%matplotlib inline

# For saving multiple plots into a single pdf file
from matplotlib.backends.backend_pdf import PdfPages 

import geopandas as gpd

from shapely.geometry import Point

from haversine import haversine

In [2]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
plutodata = gpd.read_file("../1.Transit_Access_Mobility_Score/Subway_Access_Score/nyc_mappluto_25v1_1_shp")

In [4]:
plutodata = plutodata[plutodata.is_valid]

# Filter for Manhattan only
pluto = plutodata[plutodata["Borough"] == "MN"]  

In [5]:
pluto.head()

,Borough,Block,Lot,CD,BCT2020,BCTCB2020,CT2010,CB2010,SchoolDist,Council,ZipCode,FireComp,PolicePrct,HealthCent,HealthArea,Sanitboro,SanitDistr,SanitSub,Address,ZoneDist1,ZoneDist2,ZoneDist3,ZoneDist4,Overlay1,Overlay2,SPDist1,SPDist2,SPDist3,LtdHeight,SplitZone,BldgClass,LandUse,Easements,OwnerType,OwnerName,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,GarageArea,StrgeArea,FactryArea,OtherArea,AreaSource,NumBldgs,NumFloors,UnitsRes,UnitsTotal,LotFront,LotDepth,BldgFront,BldgDepth,Ext,ProxCode,IrrLotCode,LotType,BsmtCode,AssessLand,AssessTot,ExemptTot,YearBuilt,YearAlter1,YearAlter2,HistDist,Landmark,BuiltFAR,ResidFAR,CommFAR,FacilFAR,BoroCode,BBL,CondoNo,Tract2010,XCoord,YCoord,ZoneMap,ZMCode,Sanborn,TaxMap,EDesigNum,APPBBL,APPDate,PLUTOMapID,FIRM07_FLA,PFIRM15_FL,Version,DCPEdited,Latitude,Longitude,Notes,Shape_Leng,Shape_Area,geometry
0,MN,1,10,101,1000500,10005000003,5,1000,02,1,10004,E007,1,15,8100,1,01,3B,301 COMFORT ROAD,R3-2,C4-1,None,None,None,None,GI,None,None,None,Y,Y4,08,0,X,GOVERNORS ISLAND CORPORATION,7600835,2542066,2542066,0,0,0,0,0,0,2542066,2,133,3.5,0,1,0.00,0.00,0.0,0.0,N,0,N,5,1,102535195.0,160929900.0,160929900.0,1900,0,0,Governors Island Historic District,INDIVIDUAL LANDMARK,0.33,0.5,0.0,1.0,1,1.000010e+09,0,0005,979063,190216,16a,Y,199 999,10101,None,1.000010e+09,09/20/2024,1,1,1,25v1.1,None,40.688774,-74.018704,None,0.0,7.414502e+06,"POLYGON ((980898.728 191409.779, 980881.798 19..."
1,MN,1,101,101,1000100,10001001001,1,1001,02,1,10004,E007,1,15,8200,None,None,None,1 LIBERTY ISLAND,R3-2,None,None,None,None,None,None,None,None,None,N,P7,08,0,X,U S GOVT LAND & BLDGS,541886,541886,541886,0,0,0,0,0,0,541886,2,10,0.0,0,0,500.00,1046.00,0.0,0.0,N,0,Y,5,0,4225950.0,14355900.0,14355900.0,1900,0,0,Individual Landmark,INDIVIDUAL LANDMARK,1.00,0.5,0.0,1.0,1,1.000010e+09,0,0001,971677,190636,16a,None,199 999,10101,None,0.000000e+00,None,1,None,1,25v1.1,None,40.689920,-74.045337,None,0.0,5.018973e+05,"MULTIPOLYGON (((972428.829 190679.175, 972443...."
2,MN,1,201,101,1000100,10001001000,1,1000,02,1,10004,E007,1,15,8200,1,01,None,1 ELLIS ISLAND,None,None,None,None,None,None,None,None,None,None,None,Z9,None,0,X,U S GOVT LAND & BLDGS,2764190,603130,603130,0,0,0,0,0,0,603130,2,13,0.0,0,0,0.00,0.00,0.0,0.0,N,0,N,5,0,14972400.0,111512250.0,111512250.0,1900,0,0,Ellis Island Historic District,INTERIOR LANDMARK,0.22,0.0,0.0,0.0,1,1.000010e+09,0,0001,972790,193648,12b,None,199 999,10101,None,0.000000e+00,None,1,None,1,25v1.1,None,40.698188,-74.041329,None,0.0,1.148539e+06,"POLYGON ((973648.066 193711.894, 973525.342 19..."
3,MN,2,1,101,1000900,10009001022,9,1025,02,1,10004,L015,1,15,7700,1,01,1B,4 SOUTH STREET,M1-4,None,None,None,None,None,LM,None,None,None,N,Y7,07,0,C,NYC DEPARTMENT OF SMALL BUSINESS SERVICES,209215,280000,280000,0,0,0,0,0,0,280000,2,1,5.0,0,1,314.28,564.84,268.0,210.0,N,1,Y,2,0,18818100.0,33103350.0,33103350.0,1908,2006,2006,Individual Landmark,None,1.34,0.0,2.0,6.5,1,1.000020e+09,0,0009,980670,194440,12b,None,101N100,10101,None,1.000020e+09,08/07/2012,1,1,1,25v1.1,t,40.700369,-74.012911,None,0.0,1.008250e+05,"POLYGON ((980609.55 194220.422, 980608.726 194..."
4,MN,2,2,101,1000900,10009001022,9,1025,02,1,10004,L015,1,15,7700,1,01,1B,10 SOUTH STREET,C4-6,None,None,None,None,None,LM,None,None,None,N,H5,05,0,C,10 SOUTH STREET LLC,191502,157426,10940,0,0,0,0,0,0,10940,2,1,5.0,0,2,311.61,555.84,205.0,270.0,N,0,Y,2,0,18787500.0,54858600.0,54858600.0,1900,2001,0,None,INDIVIDUAL LANDMARK,0.82,10.0,3.4,10.0,1,1.000020e+09,0,0009,981037,194506,12b,None,11N 100,10101,None,1.000020e+09,08/07/2012,1,1,1,25v1.1,None,40.700550,-74.011588,None,0.0,8.724419e+04,"POLYGON ((980854.138 194531.437, 980823.199 19..."


In [17]:
relevant_columns = ["BBL", "BCT2020",  "ZipCode", "LotArea", "BldgArea",
    "AssessTot", "UnitsRes", "LandUse", "YearBuilt", "Block", "Lot", "ZipCode", "Address",
 "ZoneDist1", "BldgClass","NumFloors",  "UnitsTotal",
    "geometry"]

In [12]:
pluto.columns

Index(['Borough', 'Block', 'Lot', 'CD', 'BCT2020', 'BCTCB2020', 'CT2010',
       'CB2010', 'SchoolDist', 'Council', 'ZipCode', 'FireComp', 'PolicePrct',
       'HealthCent', 'HealthArea', 'Sanitboro', 'SanitDistr', 'SanitSub',
       'Address', 'ZoneDist1', 'ZoneDist2', 'ZoneDist3', 'ZoneDist4',
       'Overlay1', 'Overlay2', 'SPDist1', 'SPDist2', 'SPDist3', 'LtdHeight',
       'SplitZone', 'BldgClass', 'LandUse', 'Easements', 'OwnerType',
       'OwnerName', 'LotArea', 'BldgArea', 'ComArea', 'ResArea', 'OfficeArea',
       'RetailArea', 'GarageArea', 'StrgeArea', 'FactryArea', 'OtherArea',
       'AreaSource', 'NumBldgs', 'NumFloors', 'UnitsRes', 'UnitsTotal',
       'LotFront', 'LotDepth', 'BldgFront', 'BldgDepth', 'Ext', 'ProxCode',
       'IrrLotCode', 'LotType', 'BsmtCode', 'AssessLand', 'AssessTot',
       'ExemptTot', 'YearBuilt', 'YearAlter1', 'YearAlter2', 'HistDist',
       'Landmark', 'BuiltFAR', 'ResidFAR', 'CommFAR', 'FacilFAR', 'BoroCode',
       'BBL', 'CondoNo', 'Tract2

In [ ]:
#only save relevant columns
manhatten_pluto = pluto[relevant_columns]

In [ ]:
#Derived metrics
manhatten_pluto["bldg_to_lot_ratio"] = manhatten_pluto["BldgArea"] / manhatten_pluto["LotArea"]
manhatten_pluto["assessed_value_per_sqft"] = manhatten_pluto["AssessTot"] / manhatten_pluto["BldgArea"]
manhatten_pluto["residential_density"] = manhatten_pluto["UnitsRes"] / manhatten_pluto["LotArea"]


c:\Users\sweet\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\sweet\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\sweet\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [20]:
#create Tract IDs
manhatten_pluto["tract_id"] = manhatten_pluto["BCT2020"].astype(str).str.zfill(7)


c:\Users\sweet\anaconda3\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [21]:
#export parcel features
manhatten_pluto.to_csv("manhattan_pluto_features.csv", index=False)
#manhatten_pluto.to_file("manhattan_parcels.geojson", driver="GeoJSON")
